In [3]:
import re
import time
from typing import Iterator
import requests
import lxml.html
from pymongo import MongoClient


In [4]:
def scrape_list_page(response: requests.Response) -> Iterator[str]:
    """
    一覧ページのResponseから詳細ページのURLを抜き出すジェネレーター関数
    """
    html = lxml.html.fromstring(response.text)
    html.make_links_absolute(response.url)

    for a in html.cssselect('#listBook>li>a[itemprop="url"]'):
        url = a.get('href')
        yield url


def scrape_detail_page(response: requests.Response) -> dict:
    """
    詳細ページのResponseから電子書籍の情報をdictで取得する
    """
    html = lxml.html.fromstring(response.text)
    ebook = {
        'url': response.url,
        'key': extract_key(response.url),
        'title': html.cssselect('#bookTitle')[0].text_content(),
        'price': html.cssselect('.buy')[0].text.strip(),
        'content': [normalize_spaces(h3.text_content()) for h3 in html.cssselect('#content > h3')],
    }
    return ebook

def extract_key(url: str) -> str:
    m = re.search(r'/([^/]+)$', url)
    return m.group(1)

def normalize_spaces(s: str) -> str:
    """
    連続する空白を1つのスペースに置き換え、前後の空白を削除した新しい文字列を取得する
    """
    return re.sub(r'\s+', ' ', s).strip()


In [18]:
client = MongoClient('localhost', 27017)
collection = client.scraping.ebooks
collection.create_index('key', unique=True)

'key_1'

In [19]:
session = requests.Session()
response = session.get('https://gihyo.jp/dp')
urls = scrape_list_page(response)

In [20]:
for url in urls:
    key = extract_key(url)
    
    ebook = collection.find_one({'key': key})
    if not ebook:
        time.sleep(1)
        response = session.get(url)
        ebook = scrape_detail_page(response)
        collection.insert_one(ebook)
    print(ebook)

{'_id': ObjectId('5ef5bdf7c6e5282fe56eec4a'), 'url': 'https://gihyo.jp/dp/ebook/2020/978-4-297-11491-6', 'key': '978-4-297-11491-6', 'title': 'テレワークをはじめよう', 'price': '1,340円', 'content': ['第1章 テレワークのはじめ方', '第2章 テレワークと生産性', '第3章 テレワークとルール・法律', '第4章 テレワークになかなか移行できない企業のためのヒント', '第5章 会議＆通話のテクニックとツール', '第6章 メール＆チャットのテクニックとツール', '第7章 文書・書類のテクニックとツール', 'Appidex さらに効率化するテクニック＆ツール', 'Special テレワーク企業事例インタビュー']}


DuplicateKeyError: E11000 duplicate key error collection: scraping.ebooks index: kye_1 dup key: { kye: null }

In [5]:
session = requests.Session()
response = session.get('https://gihyo.jp/dp')
urls = scrape_list_page(response)

In [8]:
urls

<generator object scrape_list_page at 0x7f105c770450>